# Redirection

### Questions:
- How can I search within files?
- How can I combine existing commands to do new things?

### Objectives:
- Employ the `grep` command to search for information within files.
- Print the results of a command to a file.
- Construct command pipelines with two or more stages.

### Keypoints:
- `grep` is a powerful search tool with many options for customization.
- `>`, `>>`, and `|` are different ways of redirecting output.
- `command > file` redirects a command's output to a file.
- `command >> file` redirects a command's output to a file without overwriting the existing contents of the file.
- `command_1 | command_2` redirects the output of the first command as input to the second command."
- `basename` gets rid of repetitive parts of names


#### Getting Started

In [ ]:
# Make sure you have the most up to date code
%cd ~/be487-fall-2024
!git stash
!git pull

In [ ]:
'''
Set a variable for your netid
Replace "MY_NETID" with your actual netid
Notice that the `cd` command is a magic command in Jupyter Notebooks and starts with %.
Run this cell to set your netid
'''
netid = "MY_NETID"
work_dir = "/xdisk/bhurwitz/bh_class/" + netid
%cd $work_dir

### Section 1: Searching files

We discussed in a previous exercise how to search within a file using `less`. We can also search within files without even opening them, using `grep`. `grep` is a command-line utility for searching plain-text files for lines matching a specific set of characters (sometimes called a string) or a particular pattern (which can be specified using something called regular expressions). We're not going to work with regular expressions in this lesson, and are instead going to specify the strings we are searching for. Let's give it a try!

#### Nucleotide abbreviations

The four nucleotides that appear in DNA are abbreviated `A`, `C`, `T` and `G`. 
Unknown nucleotides are represented with the letter `N`. An `N` appearing
in a sequencing file represents a position where the sequencing machine was not able to 
confidently determine the nucleotide in that position. You can think of an `N` as being any nucleotide at that position in the DNA sequence. 

We'll search for strings inside of our fastq files. Let's first make sure we are in the correct directory.

In [ ]:
'''
Type the commands below, and run the cell
%cd /xdisk/bhurwitz/bh_class/$netid/exercises/data/untrimmed_fastq
!ls
'''

#### What do you get?

```
JC1A_R1.fastq   JC1A_R2.fastq     JP4D_R1.fastq     JP4D_R2.fastq  TruSeq3-PE.fa
```

Great! Now, suppose we want to see how many reads in our file have really bad segments containing 10 consecutive unknown nucleotides (Ns).

#### Determining quality

In this lesson, we're going to be manually searching for strings of `N`s within our sequence
results to illustrate some principles of file searching. It can be really useful to do this
type of searching to get a feel for the quality of your sequencing results, however, in your 
research you will most likely use a bioinformatics tool that has a built-in program for
filtering out low-quality reads. You'll learn how to use one such tool in a later lesson when we look at read quality. 

Let's search for the string NNNNNNNNNN in the JC1A_R2.fastq file.

In [ ]:
'''
Type the commands below, and run the cell
!grep NNNNNNNNNN JC1A_R2.fastq
'''

#### Yikes we get a ton of output

This command returns a lot of output to the terminal. Every single line in the JC1A_R2.fastq file that contains at least 10 consecutive Ns is printed to the terminal, regardless of how long or short the file is. We may be interested not only in the actual sequence which contains this string, but in the name (or identifier) of that sequence. We discussed in a previous lesson that the identifier line immediately precedes the nucleotide sequence for each read in a FASTQ file. We may also want to inspect the quality scores associated with each of these reads. To get all of this information, we will return the line immediately before each match and the two lines immediately after each match.

We can use the `-B` argument for grep to return a specific number of lines before each match. The `-A` argument returns a specific number of lines after each matching line. Here we want the line *before* and the two lines *after* each matching line, so we add `-B1 -A2` to our grep command.

In [ ]:
'''
Type the commands below, and run the cell
!grep -B1 -A2 NNNNNNNNNN JC1A_R2.fastq
'''

#### That's better, we just get one set of lines

One of the sets of lines returned by this command is: 

```
@MISEQ-LAB244-W7:91:000000000-A5C7L:1:2111:5300:24013 2:N:0:TCGAAG
NNNNNNNNNNNCNANNANNNNNCGCCGGTGTTCTNCTGGGGNACGGANACCGAGTAGATCGGAACAGCGTCGTGGAGNGAAAGAGTGTAGATCCCGGTGGGCGGCGTATCATTAAAAAAAAAACCTGCTGGTCCTTGTCTC
+
AAA11BB3333BGG1GGEC1E?0E0B0BFDGFHD2FBH110A1BEE?A/BAFBDGH///>FEGGG><@/#//?#?/#//????########################################################################################################################################################################
```

#### Exercise 1: Using grep

1. Search for the sequence `GATCGAGAGGGGATAGGCG` in the `JC1A_R2.fastq` file.
Have your search return all matching lines and the name (or identifier) for each sequence that contains a match.


In [ ]:
'''
Type the commands below for the exercise, and run the cell
'''

#### Solution 

1. To search for the GATCGAGAGGGGATAGGCG sequence in the file JC1A_R2.fastq:

```
$ grep -B1 GATCGAGAGGGGATAGGCG JC1A_R2.fastq
```

The output shows all of the lines that contain the sequence GATCGAGAGGGGATAGGCG. As the flag -B1 is used, it also shows the previous line to each occurence. In a FastQ file the identifier of each sequence is one line avobe the sequence itself, therefore in this example you can see the names and the sequences that match your query.

## Finding what you need!

`grep` allowed us to identify sequences in our FASTQ files that match a particular pattern. All of these sequences were printed to our terminal screen, but in order to work with these sequences and perform other operations on them, we will need to capture that output in some
way. 

We can do this with something called "redirection". The idea is that
we are taking what would ordinarily be printed to the terminal screen and redirecting it to another location. In our case, we want to print this information to a file so that we can look at it later and 
use other commands to analyze this data.

The command for redirecting output to a file is `>`.

Let's try out this command and copy all the records (including all four lines of each record) in our FASTQ files that contain 
'NNNNNNNNNN' to another file called `bad_reads.txt`.

In [ ]:
'''
Type the commands below, and run the cell
!grep -B1 -A2 NNNNNNNNNN JC1A_R2.fastq > bad_reads.txt
'''

#### So what happened? Let's count the lines in our new file.

The prompt should sit there a little bit, and then it should look like nothing happened. But type `ls`. You should see a new file called `bad_reads.txt`. 

We can check the number of lines in our new file using a command called `wc`. `wc` stands for **word count**. This command counts the number of words, lines, and characters in a file. 

In [ ]:
'''
Type the commands below, and run the cell
!wc bad_reads.txt
'''

#### You should see something like this:

```
  402 489 50076 bad_reads.txt
```

This will tell us the number of lines, words and characters in the file. If we want only the number of lines, we can use the `-l` flag for `lines`.

In [ ]:
'''
Type the commands below, and run the cell
!wc -l bad_reads.txt
'''

#### what do we get?

```
402 bad_reads.txt
```

Because we asked `grep` for all four lines of each FASTQ record, we need to divide the output by four to get the number of sequences that match our search pattern.

#### Exercise 2: Using `wc`

How many sequences in `JC1A_R2.fastq` contain at least 3 consecutive Ns?

In [ ]:
'''
Type the commands below, and run the cell
'''

#### Solution

```
$ grep NNN JC1A_R2.fastq > bad_reads.txt
$ wc -l bad_reads.txt
```

```
596 bad_reads.txt
```

#### Redirecting output

We might want to search multiple FASTQ files for sequences that match our search pattern. However, we need to be careful, because each time we use the `>` command to redirect output to a file, the new output will replace the output that was already present in the file. 

This is called "overwriting" and, just like you don't want to overwrite your video recording of your kid's first birthday party, you also want to avoid overwriting your data files.

In [ ]:
'''
Type the commands below, and run the cell
!grep -B1 -A2 NNNNNNNNNN JC1A_R1.fastq > bad_reads.txt
!wc -l bad_reads.txt
'''

#### What do you get?

```
24 bad_reads.txt
```

The old `bad_reads.txt` that counts bad quality reads from file `JC1A_R2.fastq` with 402 lines has been erased.

Instead a new `bad_reads.txt` that contain 24 lines from bad reads from `JC1A_R1.fastq` has been created. We can avoid overwriting our files by using the command `>>`. `>>` is known as the "append redirect" and will append new output to the end of a file, rather than overwriting it.

```
$ grep -B1 -A2 NNNNNNNNNN JC1A_R2.fastq > bad_reads.txt
$ wc -l bad_reads.txt
```

```
402 bad_reads.txt
```

```
$ grep -B1 -A2 NNNNNNNNNN JC1A_R1.fastq >> bad_reads.txt
$ wc -l bad_reads.txt
```

```
426 bad_reads.txt
```

The output of our second call to `wc` shows that we have not overwritten our original data. The final number of 426 lines results from the adition of 402 reads from `JC1A_R2.fastq` file + 24 reads from  `JC1A_R1.fastq` file. We can also do this for more files with a single line of code by using a wildcard. 

```
$ rm bad_reads.txt
```

```
$ grep -B1 -A2 NNNNNNNNNN *.fastq >> bad_reads.txt
$ wc -l bad_reads.txt
```

```
427 bad_reads.txt
```   

In [ ]:
# Use the commands above to create a larger "bad_reads.txt" file that contains all of the bad reads for all files

#### Pipes

Since we might have multiple different criteria we want to search for, 
creating a new output file each time has the potential to clutter up our workspace. We also so far haven't been interested in the actual contents of those files, only in the number of reads that we've found. We created the files to store the reads and then counted the lines in 
the file to see how many reads matched our criteria. There's a way to do this, however, that doesn't require us to create these intermediate files - the pipe command (`|`).

This is probably not a key on your keyboard you use very much, so let's all take a minute to find that key. What `|` does is take the output that is scrolling by on the terminal and uses that output as input to another command. 
When our output was scrolling by, we might have wished we could look at it. Well it turns out that we can! We can redirect our output from our `grep` call through the `head` command.

In [ ]:
'''
Type the command below, and run the cell
!grep -B1 -A2 NNNNNNNNNN JC1A_R2.fastq | head -20
'''

In [ ]:
# The End
!cp ~/be487-fall-2024/exercises/02_intro_unix/ex02-03_redirection.ipynb  /xdisk/bhurwitz/bh_class/$netid/exercises/02_intro_unix/ex02-03_redirection.ipynb